In [2]:
import pandas as pd
from pandas.api.types import CategoricalDtype
from datetime import datetime, timedelta
from utils import format_number

In [3]:
# set parameters
LAST_N_DAYS = 30
INCENTIVE_PROGRAMS_ONLY = True

# Incentive Program Summary
Status of programs live, completed and to be announced by season.

In [4]:
df_info = pd.read_csv("inputs/" + "op_incentive_program_info" + ".csv")

if INCENTIVE_PROGRAMS_ONLY:
    df_info = df_info[df_info["Incentive / Growth Program Included?"] == "Yes"]

In [5]:
# convert to datetime
df_info["announced_date"] = pd.to_datetime( df_info["Announced On"] )
df_info["End Date"] = pd.to_datetime( df_info["End Date"] )

# convert program status into ordered categorical type
cat_size_order = CategoricalDtype(
    ["Live ‎🔥", "Coming soon ‎⏳", "Completed"], 
    ordered=True
)
df_info["Status"] = df_info["Status"].astype(cat_size_order)


# clean up for columns needed
df_info = df_info[["Source","Status","# OP Allocated","App Name","announced_date", "End Date"]]
summary = pd.pivot_table(df_info, values=["# OP Allocated", "App Name"], index=["Status", "Source"], \
    aggfunc={"# OP Allocated":"sum", "App Name":"count"})

# calculate subtotals on program status
result=pd.concat([summary,summary.groupby(level=0).sum().assign(item_name="Sub Total").set_index("item_name",append=True)]).sort_index(level=[0,1])
result = result.sort_index(level=[0, 1], ascending=[True, False])

# add grand total to summary
result.loc[("Grand Total"), "# OP Allocated"] = summary["# OP Allocated"].sum()
result.loc[("Grand Total"), "App Name"] = summary["App Name"].sum()

# cleanup display
result["# Programs"] = result["App Name"].astype(int)
result["# OP Allocated (M)"] = result["# OP Allocated"].apply(format_number)

# calculate percentage of total
result.loc[(slice(None), "Sub Total"), "# OP Allocated"] / summary["# OP Allocated"].sum()
result["% OP Allocated"] = round(result.loc[(slice(None), "Sub Total"), "# OP Allocated"] / summary["# OP Allocated"].sum() * 100).\
    astype(str).replace("\.0", "", regex=True) + "%"
result["% OP Allocated"].fillna("-", inplace=True)

result = result.replace((0, "0.0M"), "-")
display(result.drop(columns=["# OP Allocated", "App Name"]))

# Programs # OP Allocated (M)  \
Status         Source                                                
Live ‎🔥        Sub Total                     35              42.1M   
               Partner Fund                   1             450.0K   
               Governance - Season 3          -                0.0   
               Governance - Season 2          9               6.5M   
               Governance - Season 1          9               4.5M   
               Governance - Phase 0          16              30.7M   
Coming soon ‎⏳ Sub Total                     34              11.7M   
               Partner Fund                   -                0.0   
               Governance - Season 3         12               2.1M   
               Governance - Season 2         15               6.1M   
               Governance - Season 1          4               1.3M   
               Governance - Phase 0           3               2.2M   
Completed      Sub Total                      4               9.2M   
               Partner Fund                   2               8.0M   
               Governance - Season 3          -                0.0   
               Governance - Season 2          -                0.0   
               Governance - Season 1          -                0.0   
               Governance - Phase 0           2               1.2M   
Grand Total                                  73              63.0M   

                                     % OP Allocated  
Status         Source                                
Live ‎🔥        Sub Total                        67%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Coming soon ‎⏳ Sub Total                        19%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Completed      Sub Total                        15%  
               Partner Fund                       -  
               Governance - Season 3              -  
               Governance - Season 2              -  
               Governance - Season 1              -  
               Governance - Phase 0               -  
Grand Total                                       -

In [6]:
# display new programs in last 30 days
df_new_programs = df_info[df_info["announced_date"]>pd.Timestamp("today") - timedelta(days = LAST_N_DAYS)].sort_values(by="announced_date", ascending=False)
if not df_new_programs.empty:
    df_new_programs["End Date"].fillna("-", inplace=True)
    display(df_new_programs)

,Source,Status,# OP Allocated,App Name,announced_date,End Date
80,Governance - Season 2,Live ‎🔥,504000,Sushiswap,2023-03-03,-
8,Governance - Season 2,Live ‎🔥,250000,Angle,2023-02-20,-
92,Governance - Season 1,Live ‎🔥,300000,WardenSwap,2023-02-17,-
67,Governance - Season 2,Live ‎🔥,420069,Rainbow Wallet,2023-02-13,-


In [7]:
# display completed programs in last 30 days
df_completed = df_info[(df_info['Status'] == 'Completed') & (df_info['End Date']>pd.Timestamp('today') - timedelta(days = LAST_N_DAYS))]\
        .sort_values(by='announced_date', ascending=False)
if not df_completed.empty:
        display(df_completed)